In [41]:
import datetime
from io import StringIO
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from google.cloud import storage
from datetime import date
from dotenv import load_dotenv
import os
from google.oauth2 import service_account
import json 
import pandas as pd
import numpy as np


load_dotenv(override=True)

creds = os.getenv('JSON_SA_READ_WRITE_PATH')
gcp_project_id = os.getenv('GCP_PROJECT_ID')

today = date.today()
yesterday = today - datetime.timedelta(1)

In [46]:
gcp_bucket_name = os.getenv('GCP_BUCKET_NAME_STORE')
bucket_file_path = f"{yesterday}/{yesterday}.csv"

# Load service account credentials from JSON file
with open(creds) as json_file:
    data = json.load(json_file)

# Create service account credentials from JSON keyfile
credentials = service_account.Credentials.from_service_account_info(data)

# Create a Google Cloud Storage client with the provided credentials and project ID
client = storage.Client(credentials=credentials, project=gcp_project_id)

# Get the specified bucket
bucket = client.get_bucket(gcp_bucket_name)

# Get the blob (CSV file) from the bucket
blob = bucket.blob(bucket_file_path)

# Download the contents of the blob as a string
csv_content = blob.download_as_text()

# Create a Pandas DataFrame from the CSV content
df = pd.read_csv(StringIO(csv_content))

# Filter results for just those who's price change we predict 
df = df[df.price_change_this_night != 0] 

prediction_result = {"players_who_changed_price": df}

prediction_result

{'players_who_changed_price':      price_change_this_night  net_transfers_in_out_since_last_price_change  \
 128                        1                                        132315   
 531                       -1                                         -2749   
 556                        1                                        216469   
 566                        1                                        171480   
 589                       -1                                       -119586   
 
      net_transfers_in_out_this_day  price_change_so_far_for_this_event   \
 128                          33965                                    0   
 531                           -551                                    0   
 556                          62852                                    0   
 566                          44007                                    0   
 589                         -24296                                    0   
 
      total_active_players_estimate  